In [26]:
%matplotlib inline
from matplotlib import pyplot as plt
from ipycanvas import Canvas
import numpy as np
from scipy import ndimage as ndi

canvas_size = 32
brush_size = 2

canvas = Canvas(width = canvas_size, height = canvas_size, sync_image_data=True)

canvas.layout.width = '400px'
canvas.layout.height = 'auto'

drawing = False

def handle_mouse_down(x, y):
    global drawing
    drawing = True
    canvas.fill_rect(x, y, brush_size, height=None)

def handle_mouse_move(x, y):
    if drawing == True:
        canvas.fill_rect(x, y, brush_size, height=None)
    else:
        pass

def handle_mouse_up(x, y):
    global drawing
    drawing = False
    

canvas.on_mouse_down(handle_mouse_down)
canvas.on_mouse_move(handle_mouse_move)
canvas.on_mouse_up(handle_mouse_up)

canvas.stroke_style = 'red'
canvas.stroke_rect(0, 0, canvas_size, height=None)

canvas

Canvas(height=32, layout=Layout(height='auto', width='400px'), sync_image_data=True, width=32)

In [27]:
canvas_array = canvas.get_image_data(2, 2, canvas_size-4, canvas_size-4)
canvas_array = np.ravel(canvas_array)
canvas_array = canvas_array[3:len(canvas_array):4]

uncentered_canvas_array = np.reshape(canvas_array, (28, 28))

center_of_mass = ndi.measurements.center_of_mass(uncentered_canvas_array)
roll_factors = (14 - round(center_of_mass[0]), 14 - round(center_of_mass[1]))

centered_input_array = uncentered_canvas_array
centered_input_array = np.roll(centered_input_array, roll_factors[1], axis = 1)
centered_input_array = np.roll(centered_input_array, roll_factors[0], axis = 0)

nn_input_array = centered_input_array.astype('float32') / 255
nn_input_array = nn_input_array.reshape((1,784))

canvas_drawing_predition = network.predict(nn_input_array)

#np.argmax(canvas_drawing_predition)
canvas_drawing_predition

array([[2.0675680e-08, 3.0717256e-08, 2.1173006e-05, 4.1021821e-07,
        3.5702181e-01, 5.0087024e-06, 5.7768232e-01, 4.2793498e-09,
        6.5269224e-02, 8.1609698e-11]], dtype=float32)

In [1]:
from keras import models
from keras import layers
from keras import optimizers
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

network.fit(train_images, train_labels, epochs=5, batch_size=128)

Using TensorFlow backend.
2021-12-23 17:34:05.345949: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-23 17:34:05.350690: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/5
60000/60000 [==============================] - 11s 178us/step - loss: 0.2551 - accuracy: 0.9258
Epoch 2/5
60000/60000 [==============================] - 9s 156us/step - loss: 0.1043 - accuracy: 0.9688
Epoch 3/5
60000/60000 [==============================] - 9s 154us/step - loss: 0.0695 - accuracy: 0.9795
Epoch 4/5
60000/60000 [==============================] - 9s 152us/step - loss: 0.0509 - accuracy: 0.9849
Epoch 5/5
60000/60000 [==============================] - 9s 154us/step - loss: 0.0379 - accuracy: 0.9888
